# Ltlcross_runner tool settings

In [1]:
rab4 = 'Rab4/bin/'
dev_spot = '/home/xblahoud/spot_dev/bin/'

In [2]:
def ltl_to_ba(tgba=False):
    spot_n = 'Spot'
    spot_o = '' if tgba else ' -B'
    ltl3ba_n = 'LTL3BA'
    ltl3ba_o = ' -H2' if tgba else ' -H3' 
    tgba_suff = '_gen' if tgba else ''
    tools = {
        '{}{}'.format(spot_n,tgba_suff) :  \
            'ltl2tgba{} -f %f'.format(spot_o),
        '{}{}'.format(ltl3ba_n,tgba_suff) :  \
            'ltl3ba -M0{} -f %s'.format(ltl3ba_o),
        '{}d{}'.format(ltl3ba_n,tgba_suff) :  \
            'ltl3ba -M1{} -f %s'.format(ltl3ba_o),
    }
    if tgba:
        tools['LTL3TELA'] = 'ltl3tela -f %f'
    return tools

In [3]:
ltl_to_ba(True)

{'LTL3BA_gen': 'ltl3ba -M0 -H2 -f %s',
 'LTL3BAd_gen': 'ltl3ba -M1 -H2 -f %s',
 'LTL3TELA': 'ltl3tela -f %f',
 'Spot_gen': 'ltl2tgba -f %f'}

In [4]:
ltl_to_ba()

{'LTL3BA': 'ltl3ba -M0 -H3 -f %s',
 'LTL3BAd': 'ltl3ba -M1 -H3 -f %s',
 'Spot': 'ltl2tgba -B -f %f'}

In [5]:
def det_pair(det_tool, ltl_to_ba, dstar=False):
    '''produces a dictionary that combines tool for
    determinization with tools for LTL->BA translation
    
    Parameters
    ----------
    `det_tool`  : a pair of `name` and `command`
    `ltl_to_ba` : a `dict` of name->command
    `dstar`     : `Bool` that indicates whether to use ltl2dstar interface
    '''
    if not dstar:
        tools = {
            '{}+{}'.format(det_tool[0],name) :
                '{} | {} > %O'.format(cmd,det_tool[1]) for
                    name, cmd in ltl_to_ba.items()
        }
    else:
        tools = {
            '{}+{}'.format(det_tool[0],name) :
                '{} -t "{} > %%H" %L %O'.format(det_tool[1],cmd.replace('%s','%%s').replace('%f','%%s')) for
                    name, cmd in ltl_to_ba.items()
    }
    return tools

In [6]:
# name, command, generalized, dstar_interface
determinization_tools = [
    ('Spot','autfilt -DG',False, False),
    ('SpotG','autfilt -DG',True, False),
    ('devSpot',dev_spot+'autfilt -DG',False, False),
    ('devSpotG',dev_spot+'autfilt -DG',True, False),
    ('ltl2dstar','ltl2dstar -H',False, True),
    ('ltl2dstarNBA','ltl2dstar -B -H - -',False, False)
]
full_tools = {}
for tool in determinization_tools:
    full_tools.update(det_pair((tool[0],tool[1]),ltl_to_ba(tool[2]),tool[3]))

In [7]:
full_tools

{'Spot+LTL3BA': 'ltl3ba -M0 -H3 -f %s | autfilt -DG > %O',
 'Spot+LTL3BAd': 'ltl3ba -M1 -H3 -f %s | autfilt -DG > %O',
 'Spot+Spot': 'ltl2tgba -B -f %f | autfilt -DG > %O',
 'SpotG+LTL3BA_gen': 'ltl3ba -M0 -H2 -f %s | autfilt -DG > %O',
 'SpotG+LTL3BAd_gen': 'ltl3ba -M1 -H2 -f %s | autfilt -DG > %O',
 'SpotG+LTL3TELA': 'ltl3tela -f %f | autfilt -DG > %O',
 'SpotG+Spot_gen': 'ltl2tgba -f %f | autfilt -DG > %O',
 'devSpot+LTL3BA': 'ltl3ba -M0 -H3 -f %s | /home/xblahoud/spot_dev/bin/autfilt -DG > %O',
 'devSpot+LTL3BAd': 'ltl3ba -M1 -H3 -f %s | /home/xblahoud/spot_dev/bin/autfilt -DG > %O',
 'devSpot+Spot': 'ltl2tgba -B -f %f | /home/xblahoud/spot_dev/bin/autfilt -DG > %O',
 'devSpotG+LTL3BA_gen': 'ltl3ba -M0 -H2 -f %s | /home/xblahoud/spot_dev/bin/autfilt -DG > %O',
 'devSpotG+LTL3BAd_gen': 'ltl3ba -M1 -H2 -f %s | /home/xblahoud/spot_dev/bin/autfilt -DG > %O',
 'devSpotG+LTL3TELA': 'ltl3tela -f %f | /home/xblahoud/spot_dev/bin/autfilt -DG > %O',
 'devSpotG+Spot_gen': 'ltl2tgba -f %f | /h

In [8]:
sacc = ' | autfilt -S > %O'
rabinizers = {
    "R3"      : 'java -jar Rab3/rabinizer3.1.jar -silent -format=hoa -out=std %[RWM]f > %O',
    "R3_dsra" : 'java -jar Rab3/rabinizer3.1.jar -silent -format=hoa -out=std -auto=sr %[RWM]f > %O',
    "R4"      : rab4 + 'ltl2dgra %f > %O',
    "R4_dsra" : rab4 + 'ltl2dra %f' + sacc
}
ltl3dra = {
    "LTL3DRA" : 'ltl3dra -f %s > %O',
    "LTL3DRA_dsra" : 'ltl3dra -H3 -f %s > %O'
}
ltl2tgba = {
    "ltl2tgba"      : 'ltl2tgba -DG -f %f > %O',
    #"ltl2tgba_dspa" : 'ltl2tgba -DGS -f %f > %O',
    "devltl2tgba"      : dev_spot+'ltl2tgba -DG -f %f > %O',
    #"devltl2tgba_dspa" : dev_spot+'ltl2tgba -DGS -f %f > %O',
}
parity = {
    "ltl2dpa(ldba)"      : rab4 + 'ltl2dpa --mode=ldba %f > %O',
    #"ltl2dpa(ldba)_dspa" : rab4 + 'ltl2dpa --mode=ldba %f' + sacc,
    "ltl2dpa(Rab)"       : rab4 + 'ltl2dpa --mode=rabinizer %f > %O',
    #"ltl2dpa(Rab)_dspa"  : rab4 + 'ltl2dpa --mode=rabinizer %f' + sacc,
}

In [9]:
full_tools.update(rabinizers)
full_tools.update(ltl2tgba)
full_tools.update(parity)

In [10]:
ltl3dra_tools = full_tools.copy()

In [11]:
ltl3dra_tools.update(ltl3dra)

In [12]:
ltl3dra_tools

{'LTL3DRA': 'ltl3dra -f %s > %O',
 'LTL3DRA_dsra': 'ltl3dra -H3 -f %s > %O',
 'R3': 'java -jar Rab3/rabinizer3.1.jar -silent -format=hoa -out=std %f > %O',
 'R3_dsra': 'java -jar Rab3/rabinizer3.1.jar -silent -format=hoa -out=std -auto=sr %f > %O',
 'R4': 'Rab4/bin/ltl2dgra %f > %O',
 'R4_dsra': 'Rab4/bin/ltl2dra %f | autfilt -S > %O',
 'Spot+LTL3BA': 'ltl3ba -M0 -H3 -f %s | autfilt -DG > %O',
 'Spot+LTL3BAd': 'ltl3ba -M1 -H3 -f %s | autfilt -DG > %O',
 'Spot+Spot': 'ltl2tgba -B -f %f | autfilt -DG > %O',
 'SpotG+LTL3BA_gen': 'ltl3ba -M0 -H2 -f %s | autfilt -DG > %O',
 'SpotG+LTL3BAd_gen': 'ltl3ba -M1 -H2 -f %s | autfilt -DG > %O',
 'SpotG+LTL3TELA': 'ltl3tela -f %f | autfilt -DG > %O',
 'SpotG+Spot_gen': 'ltl2tgba -f %f | autfilt -DG > %O',
 'devSpot+LTL3BA': 'ltl3ba -M0 -H3 -f %s | /home/xblahoud/spot_dev/bin/autfilt -DG > %O',
 'devSpot+LTL3BAd': 'ltl3ba -M1 -H3 -f %s | /home/xblahoud/spot_dev/bin/autfilt -DG > %O',
 'devSpot+Spot': 'ltl2tgba -B -f %f | /home/xblahoud/spot_dev/bin/a

In [13]:
full_tools

{'R3': 'java -jar Rab3/rabinizer3.1.jar -silent -format=hoa -out=std %f > %O',
 'R3_dsra': 'java -jar Rab3/rabinizer3.1.jar -silent -format=hoa -out=std -auto=sr %f > %O',
 'R4': 'Rab4/bin/ltl2dgra %f > %O',
 'R4_dsra': 'Rab4/bin/ltl2dra %f | autfilt -S > %O',
 'Spot+LTL3BA': 'ltl3ba -M0 -H3 -f %s | autfilt -DG > %O',
 'Spot+LTL3BAd': 'ltl3ba -M1 -H3 -f %s | autfilt -DG > %O',
 'Spot+Spot': 'ltl2tgba -B -f %f | autfilt -DG > %O',
 'SpotG+LTL3BA_gen': 'ltl3ba -M0 -H2 -f %s | autfilt -DG > %O',
 'SpotG+LTL3BAd_gen': 'ltl3ba -M1 -H2 -f %s | autfilt -DG > %O',
 'SpotG+LTL3TELA': 'ltl3tela -f %f | autfilt -DG > %O',
 'SpotG+Spot_gen': 'ltl2tgba -f %f | autfilt -DG > %O',
 'devSpot+LTL3BA': 'ltl3ba -M0 -H3 -f %s | /home/xblahoud/spot_dev/bin/autfilt -DG > %O',
 'devSpot+LTL3BAd': 'ltl3ba -M1 -H3 -f %s | /home/xblahoud/spot_dev/bin/autfilt -DG > %O',
 'devSpot+Spot': 'ltl2tgba -B -f %f | /home/xblahoud/spot_dev/bin/autfilt -DG > %O',
 'devSpotG+LTL3BA_gen': 'ltl3ba -M0 -H2 -f %s | /home/xblah